In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/Ghadhab/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'Adaptive':0, 'Corrective':1, 'Perfective':2}
df = df.rename(columns={'msgs':'text','commit_diff':'diffs'})
df = df.replace({"labels": label2id})
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_1852/2180650394.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"labels": label2id})


repo                                    commit  \
user                                                                     
ponsonio              RxJava  0531b8bff5c14d9504beefb4ad47f473e3a22932   
ponsonio              RxJava  0950c46beda335819928585f1262dfe1dca78a0b   
ponsonio              RxJava  0f92fdd8e6422d5b79c610a7fd8409d222315a49   
ponsonio              RxJava  100f571c9a2835d5a30a55374b9be74c147e031f   
ponsonio              RxJava  191f023cf5253ea90647bc091dcaf55ccdce81cc   
...                      ...                                       ...   
jenkinsci   clearcase-plugin  51e9da224f80254476a7dc446bca817b505381d8   
jexp            batch-import  609d6c4b1eea2c33d9fb950fcbb9ba9dc1f80fc3   
hdiv                    hdiv  19b650c78a1c76f4fd90274d7f163f863c0d39e4   
casidiablo       persistence  d7bf95159df37a3d338ca267dddd3d26b38ec37c   
jhy                    jsoup  d6fd1637307f6b088eb226c3f979085725530f32   

            labels                                               text  \
user                                                                    
ponsonio         2              Change hasException to hasThrowable--   
ponsonio         0  Trying to extend the Scheduler interface accor...   
ponsonio         0   RunAsync method for outputting multiple values--   
ponsonio         1  forEach with Action1 but not Observer--I re-re...   
ponsonio         1  1.x: Fix Completable swallows- OnErrorNotImple...   
...            ...                                                ...   
jenkinsci        2  Use a temporary file to decrease memory consum...   
jexp             2  added a more memory efficient structure for st...   
hdiv             2               Memory and performance optimizations   
casidiablo       2  Now it is possible to specify the sqlite open ...   
jhy              2            Performance improvment for Element.text   

                                                        diffs  \
user                                                            
ponsonio    diff --git a/rxjava-core/src/main/java/rx/Noti...   
ponsonio    diff --git a/rxjava-core/src/main/java/rx/Sche...   
ponsonio    diff --git a/rxjava-contrib/rxjava-async-util/...   
ponsonio    diff --git a/language-adaptors/rxjava-groovy/s...   
ponsonio    diff --git a/src/main/java/rx/Completable.java...   
...                                                       ...   
jenkinsci   diff --git a/src/main/java/hudson/plugins/clea...   
jexp        diff --git a/src/main/java/org/neo4j/batchimpo...   
hdiv        diff --git a/hdiv-config/src/main/java/org/hdi...   
casidiablo  diff --git a/pom.xml b/pom.xml\nindex 394263b....   
jhy         diff --git a/src/main/java/org/jsoup/helper/St...   

                                                      feature  
user                                                           
ponsonio    [1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
ponsonio    [2, 44, 0, 0, 30, 0, 0, 1, 18, 0, 0, 0, 0, 0, ...  
ponsonio    [2, 53, 0, 0, 42, 0, 0, 1, 45, 1, 0, 0, 0, 0, ...  
ponsonio    [1, 5, 122, 9, 10, 9, 4, 1, 5, 18, 2, 0, 0, 0,...  
ponsonio    [1, 1, 0, 0, 0, 0, 0, 1, 21, 0, 0, 0, 0, 0, 0,...  
...                                                       ...  
jenkinsci   [2, 12, 0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...  
jexp        [10, 159, 29, 35, 9, 2, 1, 5, 106, 0, 4, 8, 0,...  
hdiv        [31, 302, 131, 140, 170, 89, 53, 7, 88, 14, 17...  
casidiablo  [5, 57, 20, 9, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
jhy         [3, 11, 11, 9, 13, 0, 0, 1, 0, 8, 0, 0, 1, 0, ...  

[1781 rows x 6 columns]

In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
train = train.groupby("labels", group_keys=False).apply(lambda x: x.sample(n=5, random_state=42))

/tmp/ipykernel_1852/878631650.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = train.groupby("labels", group_keys=False).apply(lambda x: x.sample(n=5, random_state=42))


In [5]:
# 假设你有一个DataFrame df，其中包含'commit_message'和'label'列
# df = pd.DataFrame({'commit_message': [...], 'label': [...]})

# 1. 数据预处理和特征提取
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['diffs'])
y_train = train['labels']

# 3. 训练XGBoost模型
clf = XGBClassifier()
clf.fit(X_train, y_train)

# 4. 评估模型
y_pred = clf.predict(vectorizer.transform(test['diffs']))
print("Accuracy:", accuracy_score(test['labels'], y_pred))
print("Classification Report:\n", classification_report(test['labels'], y_pred,digits=4))


Accuracy: 0.3917910447761194
Classification Report:
               precision    recall  f1-score   support

           0     0.3895    0.4684    0.4253        79
           1     0.3636    0.5393    0.4344        89
           2     0.4878    0.2000    0.2837       100

    accuracy                         0.3918       268
   macro avg     0.4136    0.4026    0.3811       268
weighted avg     0.4176    0.3918    0.3755       268

